<a href="https://colab.research.google.com/github/NUELBUNDI/Work-Automation-Scripts/blob/main/CreditCardReconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
df=pd.read_csv("Equitycard.csv")
df2=pd.read_csv("B303050.csv")
absa=pd.read_csv("Absa.csv")
absa_pos_code=pd.read_csv("absa_pos_branch_code.csv")

In [ ]:
def format_bank_entries():
    df.columns=df.columns.str.replace(' ','')
    df.drop(columns=['Settlementdate','Accountnumber','Merchantname','TRANS_TYPE','Cashback','Cashbackfee','Cashbackcomm','Currency','Rrn','Tid','Siccode'],inplace=True)
    # df.dropna(inplace=True)
    df['check']=df['Cardnumber'].str[-4:]
    # df['Postingdate']=pd.to_datetime(df['Postingdate'])
    # df['date']=df['Transdate'].dt.date
    df.rename(columns={'Transamount':'amount'},inplace=True)
    df['amount']=df['amount'].str.replace(',','').astype('float')
    return df
df=format_bank_entries()

In [ ]:
def format_bc_entries():  
    df2.columns=df2.columns.str.replace(' ','')
    df2.drop(columns=['SourceCode','DocumentType',\
        'DepartmentCode','Amount','DimensionSetID','Bal.AccountNo.',\
            'Bal.AccountType','BankAccountNo.','Description','ExternalDocumentNo.','DrawerName','Open','EntryNo.'],inplace=True)
    df2['check']=df2['CreditCardNo.'].str[-4:]
    df2.rename(columns={'Amount(LCY)':'amount'},inplace=True)
    df2['Amount']=df2['amount']
    df2['amount']=df2['amount'].str.replace(',','').astype('float')
    return df2

df2=format_bc_entries()

In [ ]:
def format_absa_bank_entries():
    absa.columns=absa.columns.str.replace(' ','')
    absa.drop(columns=['LOCATION_NO','LEGAL_NAME','PROCESSING_DATE','PAYMENT_DATE','TERMINAL_ID',\
                    'AUTH_ID','TRXN_TYPE','CURRENCY','PMNT_TYPE','TRXN_SOURCE','SCHEME',\
                        'COMMERCIAL_NAME','ARN_REFERENCE','RETRIEVAL_REF_NO','TIP_AMOUNT',\
                        'CARD_PRESENT' ],inplace=True)
    absa['check']=absa['CARD_NO'].str[-4:]
    absanew=pd.merge(absa,absa_pos_code,how='left',on=['ACCOUNT_NO'])
    absanew.rename(columns={'AMOUNT':'amount'},inplace=True)
    absanew['Amount']=absanew['amount']
    absanew['amount']=absanew['amount'].replace(',','').astype('float')
    return absanew
absanew=format_absa_bank_entries()

In [ ]:
equity_reconciliation=pd.merge(df,df2,how='left',on=['check','amount'])
equity_reconciliation.to_csv('equity_reconciliation.csv')

In [ ]:
absa_reconcilation=pd.merge(absanew,df2,how='left',on=['check','amount'])

absa_reconcilation.to_csv('absa_reconcilation.csv')